In [1]:
import os

In [2]:
%pwd

'c:\\Users\\DELL\\ML-Workflow-with-DVC-Pipelines\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\DELL\\ML-Workflow-with-DVC-Pipelines'

In [6]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir :Path
    source_URL : str
    Local_data_file :Path
    unzip_dir :Path

In [ ]:
from chicken_disease_detection.constants import *
from src.chicken_disease_detection.utils.common import read_yaml, create_directories


In [17]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath: str = CONFIG_FILE_PATH,
        params_filepath: str = PARAMS_FILE_PATH
    ):
        # Read YAML files
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        # Create root artifact directory
        create_directories([self.config.artifact_dir_root])

    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config